# Identifying faces

> we identify faces

- toc: false 
- comments: true
- categories: [python]

In [ ]:
!pip install torch torchvision
!pip install torchviz 
!pip install --upgrade torch torchvision

# the Pandas mismatch hasn't been a problem, because we aren't using it in our code
#   ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 1.1.0 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()

In [1]:

# Websites for Binary classification on Images
# https://github.com/jayrodge/Binary-Image-Classifier-PyTorch
# https://github.com/jayrodge/Binary-Image-Classifier-PyTorch/blob/master/Binary_face_classifier.ipynb
# https://hackernoon.com/binary-face-classifier-using-pytorch-2d835ccb7816 - this website explains the following code


import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler


# import torch.nn as nn
# import torch.optim as optim
# import torch.utils.data as data
# import torchvision.models as models
# # from torch.autograd import Variable
# from torchvision.models.vgg import model_urls
# from torchviz import make_dot

# The following program is obtained from:
# https://github.com/jayrodge/Binary-Image-Classifier-PyTorch/blob/master/Binary_face_classifier.ipynb
#
# Visualization tool
# https://stackoverflow.com/questions/52468956/how-do-i-visualize-a-net-in-pytorch 



from google.colab import drive
from google.colab import files
from PIL import Image

drive.mount('/content/gdrive')

#########################################################
# Processing the Dataset
#-------------------------
# You might often need to process the image data before passing it to the model.
# For example, if the sizes of all images are different (which often the case in large datasets), 
# then your model will throw an error. So resize them, and you can consider rescaling the pixel values also.
# Apart from this, you can perform diverse transforms for data augmentation.
# An elegant way to apply multiple transforms is using transform.compose() as shown below.
#
# When comes to loading/ preprocessing the data PyTorch is much simpler as compared to other libraries. 
# However, PyTorch has a built-in function called transforms using which you can perform all your pre-processing tasks 
##############################################################

# how many samples per batch to load
batch_size = 16

# percentage of training set to use as validation
test_size = 0.3 # test size is 30% of the total images that means training size is 70% of the total images
valid_size = 0.1 # validation is 10% of the training image; validation comes before testing and validates each input


ImageWidth=384
ImageHeight=384

#the most common choices for convolution filter kernel sizes appear to be square shape of sizes 3X3, 5X5
#smaller sized kernel allows for more granular information. Here 5X5 is used.
#Maxpool is needed when the image is larger. It keeps the item with the maximum value.
convKernelSize1 =5
convKernelSize2 =5
maxpoolKernelSize = 2

#conv1 Input Channel is defalut 3 for R, G, B channels directly from the colored Image
convOutputChannel1 = 16
#convInputChannel2 is the same as the convOutputChannel1
convOutputChannel2 = 32
linearOut1 = 256
linearOut2 = 84

dropOutValue = 0.2
#------------------------------------------------------------------------


# for the first convolution and max pool
height1=int((ImageHeight - convKernelSize1 + 1)/maxpoolKernelSize) 
width1=int((ImageWidth - convKernelSize1 + 1)/maxpoolKernelSize)

# for the second convolution and max pool
height2=int((height1 - convKernelSize2 + 1)/maxpoolKernelSize) 
width2=int((width1 - convKernelSize2 + 1)/maxpoolKernelSize)

print(" ImageWidth = ", ImageWidth)
print(" ImageHeight = ", ImageHeight)
print(" width1 = ", width1)
print(" height1 = ", height1)
print(" width2 = ", width2)
print(" height2 = ", height2)

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize(size=(ImageWidth,ImageHeight)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # need to keep these transforms, at least do resize all images to same size and co
    ])

###########################################################################


#------------------------------------------------------------------------

data_directory_face = '/content/gdrive/MyDrive/DNN_ML/face'

print("\n\n ****************  START LOADING DATA AND PROCESSING ******************* ")

###########
# Select one of the above data_directory_XYZ
# Depending on what you want to run. 
# You can change this part for different tests
############

data = datasets.ImageFolder(data_directory_face,transform=train_transform) 
# data = datasets.ImageFolder(data_directory_tumor)

# The data needs to be split in Train, Test and validation set before training.
# - Train set will be used to train the model.
# - Validation set will be used for validating the model after each epoch. 
# - Test set will be used to evaluate the model once it is trained.


num_data = len(data)
print("num_data = ", num_data)

indices_data = list(range(num_data))
np.random.shuffle(indices_data)

#For test and training
#------------------------------
split_tt = int(np.floor(test_size * num_data))
train_idx, test_idx = indices_data[split_tt:], indices_data[:split_tt]

print("\n train_idx = ", train_idx)
print("\n test_idx = ", test_idx)

#From Training separate something For validation (for each epoch)
#---------------------------------------------------------------
num_train = len(train_idx)
indices_train = list(range(num_train))
np.random.shuffle(indices_train)
split_tv = int(np.floor(valid_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

print("\n\n train_new_idx = ", train_new_idx)
print("\n valid_idx = ", valid_idx)

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_new_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_idx)



# Loaders contains the data in tuple format (Image in form of tensor, label)
train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, sampler=train_sampler, num_workers=1)

# only running the data augmentation on the training data, double check if works
valid_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, sampler=valid_sampler, num_workers=1)
test_loader = torch.utils.data.DataLoader(data, sampler = test_sampler, batch_size=batch_size, num_workers=1)

# variable representing classes of the images
classes = [0,1] # labeling either 0 (negative) or 1 (positive)

total_length = len(test_loader)*batch_size + len(valid_loader)*batch_size + len(train_loader)*batch_size
print("total_length = ", total_length)

for batch in valid_loader:
    print("batch[0].size() = ", batch[0].size())


import matplotlib.pyplot as plt
%matplotlib inline

# helper function to un-normalize and display an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image
    
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = next(dataiter)
images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(10, 4))

print(" ------------------ display images ------------------------")
# display some images with Labels to see if labeling appears correct


no_of_image_to_display = 10
for idx in np.arange(no_of_image_to_display):
    ax = fig.add_subplot(2, no_of_image_to_display/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])

display(plt.show())
plt.close()



# Building the Model: Decrypting the layers
# ---------------------------------------------
# The model class definition should always have __init__() method. In this, we will initialize the blocks/layers and other parameters.
# Talking about the neural network layers, there are 3 main types in image classification: convolutional, max pooling, and dropout .

# Convolution layers:  Convolutional layers will extract features from the input image and generate feature maps/activations. 
# You can decide how many activations you want using the filters argument. 
# Basically, when you apply convolution upon an image, the kernel will pass over the entire image in small parts, 
# and it will give an activation. It is demonstrated in the below image.

# Pooling Layers
# When we use conv2d layers, we end up with a lot of feature maps that occupy high computational space. 
# In order to decrease the computational time and space required, you can use Max pooling or average pooling.
# For each patch/group of the map, only the maximum is chosen to form the output. 
# The below image clearly demonstrates the work.

# Dropout Layers
# You can guess its function from the name itself! It drops out like 10-20% of the data.
# Overfitting is a common problem when you are training over the same data for many iterations/epochs. 
# By dropping out a randomly chosen 10% of data, the model will be able to generalize more to new data.
# This concludes with a brief description of the layers we have used in our code. 
# Note that the final layer has output as 2, as it is binary classification.

Mounted at /content/gdrive
 ImageWidth =  384
 ImageHeight =  384
 width1 =  190
 height1 =  190
 width2 =  93
 height2 =  93


 ****************  START LOADING DATA AND PROCESSING ******************* 
num_data =  400

 train_idx =  [233, 261, 54, 14, 246, 47, 309, 360, 215, 189, 131, 22, 252, 25, 186, 164, 66, 253, 351, 84, 33, 280, 155, 334, 133, 137, 239, 90, 3, 288, 219, 106, 160, 234, 313, 210, 172, 220, 74, 180, 173, 248, 391, 389, 8, 35, 70, 62, 130, 369, 39, 107, 328, 376, 48, 96, 310, 312, 316, 78, 322, 135, 16, 81, 147, 355, 341, 191, 365, 42, 257, 188, 176, 283, 317, 128, 352, 153, 41, 353, 386, 15, 304, 17, 20, 179, 2, 238, 320, 91, 292, 77, 113, 354, 331, 291, 300, 27, 101, 152, 4, 229, 80, 242, 230, 86, 209, 255, 357, 208, 34, 75, 104, 5, 375, 216, 293, 221, 166, 398, 139, 149, 314, 59, 199, 350, 387, 228, 323, 125, 298, 315, 264, 67, 60, 397, 26, 218, 142, 19, 124, 192, 185, 243, 311, 196, 79, 49, 203, 279, 61, 36, 69, 336, 284, 390, 281, 115, 269, 213, 295, 324, 382, 6,

ValueError: ignored

<Figure size 1000x400 with 0 Axes>